# Business Analytics : TP6

## Création d'un RAG : Chatbot avec capacité à rechercher dans la base de donné ce qu'il a besoin.

Pour ça on va utiliser le dataset : https://github.com/luminati-io/Amazon-dataset-samples qui est un sample de la base de données d'Amazon.

Le chatbot va devoir :
- Comprendre la requête du client en extrayant les informations importantes
- Demander plus d'information si necessaire
- Demander pour clarifier si quelque chose n'est pas claire
- Donner des recomendations importante

Idée c'est pouvoir chercher pour un type de produit avec un prix maximum.


<!-- Ce que je voudrais faire :
- avoir un prompt de raisonement au dessus que reviens à lui plusieurs fois et qui peut faire:
    - extraire informations
    - rechercher si suffisament d'information
    - répondre par LLM pour demander plus d'information ou répondre à l'utilisateur
- avoir fonction extraction information en json
- avoir fonction de recherche dans la base de données
- avoir fonction pour générer text pour expliquer que besoin info ou répondre à la question -->


In [21]:
import pandas as pd
from transformers import AutoTokenizer, pipeline, LlamaForCausalLM
import torch
import json
from sentence_transformers import SentenceTransformer, util, CrossEncoder
from time import time
import numpy as np
import os
from heapq import nlargest
from IPython.display import Markdown, display #print joly en markdown
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Chargement des données

On voit en chargeant des donné que les prix sont en plusieur unité différentes.

In [2]:
dataset = pd.read_csv("./amazon-products.csv")

# nettoie et suprime les " qui faut pas 
dataset['initial_price'] = dataset['initial_price'].str.replace('"', '', regex=False)
dataset['final_price'] = dataset['final_price'].str.replace('"', '', regex=False)

#cast en float
dataset['initial_price'] = pd.to_numeric(dataset['initial_price'], errors='coerce')
dataset['final_price'] = pd.to_numeric(dataset['final_price'], errors='coerce')

#supprime doublons
dataset.drop_duplicates(inplace=True)

dataset.head()

,timestamp,title,seller_name,brand,description,initial_price,final_price,currency,availability,reviews_count,...,root_bs_category,bs_category,bs_rank,badge,subcategory_rank,amazon_choice,images,product_details,prices_breakdown,country_of_origin
0,2023-08-08 00:00:00.000,Saucony Men's Kinvara 13 Running Shoe,Orv███tor███,Saucony,"When it comes to lightweight speed, nothing cr...",NaN,57.79,USD,In Stock,702,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-08-09 00:00:00.000,Kishigo Premium Black Series Heavy Duty Unisex...,Ama███.co███,Kishigo,The Kishigo Premium Black Series Heavy Duty Ve...,NaN,28.50,USD,In Stock,916,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-02-04 00:00:00.000,TWINSLUXES Solar Post Cap Lights Outdoor - Wat...,Twi███uxe███,TWINSLUXES,Solar Post Cap Lights Waterproof LED Fence Pos...,49.99,33.99,USD,In Stock,3178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-09 00:00:00.000,Accutire MS-4021B Digital Tire Pressure Gauge ...,Cit███ran███Dir██████,Accutire,About this item Heavy duty construction and ru...,17.95,17.95,USD,In Stock,8034,...,Automotive,Tire Repair Tools,50.0,NaN,"[{""subcategory_name"":""Automotive"",""subcategory...",False,NaN,NaN,NaN,NaN
4,2024-01-16 00:00:00.000,SAURA LIFE SCIENCE Adivasi Ayurvedic Neelgiri ...,PRA███ EN███PRI███,SAURA LIFE SCIENCE,This extraordinary fusion is designed to nouri...,1299.00,799.00,INR,In stock,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
print(dataset['currency'].unique())

['USD' 'INR' 'GBP']


On voit qu'on a des £ \$ et INR, par simplicité on peut dire que 0.74£ = 1\$ par contre 1\$ = 90 INR. va update le dataset pour ça.

In [4]:
#remplace GDB par usd et mutliply par 0.74
dataset.loc[dataset['currency'] == 'GDB', 'final_price'] = dataset.loc[dataset['currency'] == 'GDB', 'final_price'].astype(float) * 0.74
dataset['currency'] = dataset['currency'].replace('GDB', 'USD')

# pareil pour INR
dataset.loc[dataset['currency'] == 'INR', 'final_price'] = dataset.loc[dataset['currency'] == 'INR', 'final_price'].astype(float) * 90
dataset['currency'] = dataset['currency'].replace('INR', 'USD')

Peut certainement prétraité data mais comme veut que nom et prix et que nom déjà un string alors va pas se tracasser plus

juste faudra trouver dans description et title maybe peut être faire un bi-encodeur pour meilleur RAG à voir je vois l'utilité pour rechercher mieux information. Va surtout se concentrer sur le fonctionnement RAG ici.

### Chargement modèle

Pour le modèle, on a finetune le modèle la fois dernière. Il a certe apris la manière d'apprendre mais aussi un peu des informations de l'ancien dataset. En plus, on a la finetune pour mieux écrire pas respecter de l'écriture en JSON comme on va commence par devoir extraire. Préfère utilisé au début un modèle normal pour faire cette extraction et après passé sur le finetuné si tout va bien.

In [5]:
model_name = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = LlamaForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,
    device_map="auto",
    load_in_8bit=False,
    load_in_4bit=True,
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 1er étape du pipeline
La première étape du pipeline va d'êxtraire depuis la question un JSON ayant le prix, une description, des keyword si présente et missing_infromation.

Je ne vais pas utilisé pipeline comme prof utilise car n'est pas adapté, vaut mieux utilisé generate comme fait au TP1, ça fait que le modèle est beaucoup moins génératif et suit mieux les consignes ce qui donnera de meilleur résultat pour ici.

In [6]:
def extract_json(text, model, tokenizer):
    messages = [
    {
        "role": "system",
        "content": (
        "You are an e-commerce assistant. "
        "Given a user query, extract the following fields and return JSON only:\n"
        "- item_description: a short description of what the user wants\n"
        "- max_price: numeric value if provided, otherwise null\n"
        "- keywords: list of optional keywords\n"
        "- missing_information: list of required fields not provided by the user\n"
        "Required fields: item_description, max_price.\n"
        "Only return the JSON so that it is parsable using json.loads(response). Do not add any other text or information."
        )
    },
    {"role": "user", "content": text}
    ]

    #fait à la main
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    

    while True:
        outputs = model.generate(
            **inputs,
            max_new_tokens=80,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
        )

        response = tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)

        try:
            return json.loads(response)
        except json.JSONDecodeError:
            print("Invalid JSON, retring, regenerating solution:", response)

In [8]:
user_query = "I am looking for wireless headphones under 80 euros. I would like to have a bose with noise cancelling if possible."
extracted_data = extract_json(user_query, model, tokenizer)
print(extracted_data)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


{'item_description': 'wireless headphones', 'max_price': 80, 'keywords': ['wireless', 'noise cancelling', 'bose'], 'missing_information': ['brand', 'model', 'color']}


### Partie 2 recherche dans le dataset
Pour la partie recherche d'information, on va uiliser un système de retrieve and rerank utilisant un système de bi-encodeur pour une 1er recherche suivi d'un cross encodeur pour affiner la recherche.

In [7]:
def get_top_k(model, embed_corpus, corpus, train_queries, top_k=100, timeRequested=False):
    startTime = time()

    query_embeddings = model.encode(list(train_queries.values()), show_progress_bar=True, batch_size=64, convert_to_numpy=True)

    hits = util.semantic_search(query_embeddings, embed_corpus, top_k=top_k)

    corpus_ids = list(corpus.keys())

    query_ids = list(train_queries.keys())

    # Format results
    retrieval_results = {}
    for qid, query_hits in zip(query_ids, hits):
        retrieval_results[qid] = {
            corpus_ids[hit['corpus_id']]: float(hit['score']) for hit in query_hits
        }
    totalTime = time() - startTime
    if timeRequested:
        print(f"A pris {totalTime:.2f} secondes pour récupérer les top {top_k} documents pour {len(train_queries)} requêtes. correspondant à {totalTime/len(train_queries):.4f} secondes par requête.")
    return retrieval_results

def limit_length_tokens(text, tokenizer, max_tokens=510):
    """
   fonciton qui coupe tout le text à un max token
    """
    if text is None or text == "": #sécurité si texte vide
        return ""
    
    
    text = str(text) #force text
    
    # mets en token et cut
    tokens = tokenizer.encode(
        text, 
        add_special_tokens=False,  # évite token [CLS] ou autres que pas besoin
        truncation=True, 
        max_length=max_tokens
    )

    
    # decode pour revenir au text mais tronqué
    truncated_text = tokenizer.decode(tokens, skip_special_tokens=True)
    
    return truncated_text

model_retrieve = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", device="cpu") #force cpu sinon manque 100Mo de ram
model_cc = CrossEncoder("cross-encoder/mmarco-mMiniLMv2-L12-H384-v1", device="cpu") #charge model cross encoder entrainé
tokenizer_cc = AutoTokenizer.from_pretrained("cross-encoder/mmarco-mMiniLMv2-L12-H384-v1")

def retrieve_and_rerank(corpus, queries, top_k_retrieve=100, top_k_rerank=5, requestedTime = False, embed_corpus=None): #premier fonction qui permet de faire 1er étape qui cherche top k et puis le passe dans la seconde méthode
    '''
        corpus: dict doc_id -> doc_text
        queries: dict query_id -> query_text
        retrieve_fn: fonction pour trouver les top k documents pour donner à la seconde partie
        rerank_fn: fonction pour reranker les documents donnés par retrieveFn
        top_k_retrieve: nombre de documents à récupérer dans la première étape
        top_k_rerank: nombre de documents à récupérer dans la seconde étape
    '''

    assert top_k_retrieve >= top_k_rerank, "tok_k retrieve doit être supérieur ou égal à top_k_rerank sinon fait pas de sens pour la méthode de reranking"


    #commence par embed le corpus si pas déjà fait
    if embed_corpus is None:
        embed_corpus = model_retrieve.encode(list(corpus.values()), show_progress_bar=False, batch_size=64, convert_to_numpy=True)

    startTime = time()
    retrieve_res = get_top_k(model_retrieve, embed_corpus, corpus, queries, top_k=top_k_retrieve) #utilise fonction get_top_k définie plus haut pour avoir top k du retrieve

    retrieve_result = {}
    retrieve_result_first_stage = {}

    #pour chaque va retraité
    for qid, ranking in retrieve_res.items():
        ranked_corpus = {id: corpus[id] for id in ranking.keys()} #limite le corpus à ce qui a été rémonté avant
        
        pairs = [(limit_length_tokens(queries[qid], tokenizer, max_tokens=250), limit_length_tokens(doc_text, tokenizer, max_tokens=250)) for doc_text in ranked_corpus.values()] #crée pari question, text pour passer dans cross encoder limite aussi la longueur pour pas planter
        

        scores = model_cc.predict(pairs) #encode tout

        # créee dico pour valeur avec chaque
        reranked_docs = {
            doc_id: float(score) for doc_id, score in zip(ranked_corpus.keys(), scores)
        }
        # garder le top k du reant
        top_reranked = dict(
            nlargest(top_k_rerank, reranked_docs.items(), key=lambda item: item[1])
        )
        retrieve_result[qid] = top_reranked


    
    if requestedTime:
        print(f"A pris {time() - startTime:.2f} secondes pour reranker les top {top_k_retrieve} documents pour {len(queries)} requêtes avec une moyenne de {(time() - startTime)/len(queries):.4f} secondes par requête.")
    return retrieve_result, retrieve_res

#load embed corpus si existe sinon le créer
if not os.path.exists("embed_corpus.npz"):
    embed_corpus = model_retrieve.encode(list(dataset['title']), show_progress_bar=True, batch_size=64, convert_to_numpy=True)
    np.savez("embed_corpus.npz", embed_corpus=embed_corpus)
else:
    embed_corpus = np.load("embed_corpus.npz")["embed_corpus"]

In [11]:
resultat, _ = retrieve_and_rerank(dataset['title'].to_dict(), {"query1": "wireless bluetooth headphones"}) #fonction prévu pour faire du batch donc peut être à revoir mais bon récupère d'un autre projet et sait qu'elle fonctionne bien

print(resultat)

for doc_id, score in resultat['query1'].items():
    print(f"Doc ID: {doc_id}, titel {dataset.loc[int(doc_id), 'title']}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'query1': {794: 9.362723350524902, 944: 7.877621173858643, 638: 7.585773944854736, 11: 6.901961326599121, 494: 1.4186797142028809}}
Doc ID: 794, titel CXK Bluetooth Headphones Wireless Earbuds 52H Playtime Bluetooth 5.3 Neckband Bluetooth Wireless Headphones, IPX7 Waterproof HiFi Deep Bass Earphones with USB-C Fast Charging for Sport Running Gym
Doc ID: 944, titel Wireless Earbud, 2023 Sport Wireless Bluetooth 5.3 Earbud with HiFi Stereo, 75H Wireless Headphones with Noise Cancelling Mic, IP7 Waterproof Bluetooth Earphones, LED Display, Button Control, White
Doc ID: 638, titel Wireless Earbuds,Bluetooth 5.3 Headphones Build in Noise Cancelling, Bluetooth Earbuds With LED Power Display, Hi-Fi Stereo, Touch Control, Waterproof/Sweatproof Wireless Headphones for iOS/Android
Doc ID: 11, titel Trucker Bluetooth Headsets, Wireless Headset with AI Environmental Noise Cancelling & Mute Microphone, Up to 30H Talk Time, 164ft Wireless Range, Bluetooth Over Ear Headphones for PC, Computer, Skype

### 3eme partie : appel LLM pour répondre à la question

In [18]:
def writte_message(message, model, tokenizer, products_list):

    prod_txt = ""
    for product in products_list:
        prod = dataset.loc[int(product), :] #prend tout la ligne du produit
        prod_txt += f"- Title: {prod['title']}\n Final Price: {prod['final_price']} {prod['currency']}\n   Description: {prod['description']}\n\n"

    # print(prod_txt)

    messages = [ 
        {
            "role": "system",
            "content": (
                "## Instructions :\n"
                "You are an e-commerce assistant. \n"
                "You will receive a user message with some products.\n"
                "The customer is asking for a maximum price. You will need to filter all product with this maximum price and if there is none, give the cheapest one.\n"
                "You will then suggest products that match the user's request.\n"
                "Remain always polite and friendly and concise.\n"

                "\n\n## Products :\n"
                f"{prod_txt}"

            )
        },
        {"role": "user", "content": message}
    ]

    #fait à la main
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,
        eos_token_id=tokenizer.eos_token_id,
    )

    return tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True) #retourne le texte généré

### Le tout mis ensemble

In [28]:
def ask_question(user_question, model, tokenizer, embed_corpus):
    #extrait les infos de la question
    extracted_data = extract_json(user_question, model, tokenizer)
    # print("[LOG] Extracted Data:", extracted_data)

    if extracted_data['item_description'] is None or extracted_data['max_price'] is None:
        return "Sorry, I need more information to help you. Please provide a description of the item and your maximum price."

    #récupère les produits en fonction de la description et des mots clés
    query_text = extracted_data['item_description']
    if extracted_data['keywords']:
        query_text += " " + " ".join(extracted_data['keywords'])

    retrieve_res, _ = retrieve_and_rerank(
        dataset['title'].to_dict(),
        {"user_query": query_text},
        embed_corpus=embed_corpus
    )

    retrieved_products = list(retrieve_res['user_query'].keys())
    # print("[LOG] Retrieved Products IDs:", retrieved_products)

    #génère la réponse avec les produits récupérés
    response = writte_message(user_question, model, tokenizer, retrieved_products)
    return response

### Test

In [22]:
Question = "I want to buy wireless headphones, preferably Bose with noise cancelling, and my budget is 80 USD."
print("User Question:", Question)
display(Markdown(ask_question(Question, model, tokenizer, embed_corpus)))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


User Question: I want to buy wireless headphones, preferably Bose with noise cancelling, and my budget is 80 USD.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Based on your preferences, I've searched for affordable wireless headphones with noise cancellation within your budget of $80.

Unfortunately, we couldn't find any exact Bose model under $80 with active noise cancellation. However, I found a few options that may interest you:

* Anker Soundcore Space Q45 Wireless Headphones: These headphones offer good noise cancellation and sound quality for around $69.99 USD. They're also foldable, making them easy to store and transport.
* Edifier H840 Wireless Noise Canceling Headphones: These headphones boast impressive noise cancellation and sound quality, priced at approximately $59.99 USD. They also come with a carrying pouch and are designed for everyday use.
* TaoTronics TT-BH040 Wireless Headphones: Another option in the market, these headphones offer decent noise cancellation and sound quality for around $49.99 USD. They also come with a carry bag and are designed for general use.

While they might not be exactly what you were looking for, these alternatives should still meet your requirements and fit your budget!

Would you like me to recommend any specific brand or style among these? Or would you prefer me to continue searching for something closer to your original preference (Bose)?

test sans le prix.

In [29]:
Question = "I want to buy wireless headphones, preferably Bose with noise cancelling"
print("User Question:", Question)
display(Markdown(ask_question(Question, model, tokenizer, embed_corpus)))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


User Question: I want to buy wireless headphones, preferably Bose with noise cancelling


Sorry, I need more information to help you. Please provide a description of the item and your maximum price.

### Résultat

On voit que le modèle recherche bien et permet de conseiller, il faudrait cependant créer un système d'historique et faire du multiturn pour qu'il soit vraiment utilisable. Cependant, pour un premier jet il est pas mal je trouve.